This Notebook is a brief sketch of how to use Grover's algorithm.
We start by declaring all necessary imports.

In [ ]:
from pyquil.api import SyncConnection
from itertools import product
from mock import patch

import numpy as np

from grove.amplification.grover import Grover

Grover's algorithm can be used to amplify the amplitude of an oracle-selected bitstring in an input superposition state. Below we construct a bitstring_map to select a target_bitstring.

In [ ]:
target_bitstring = '010'
bit = ("0", "1")
# The way the oracle should mark the target_bitstring is by imparting a phase of -1.
# It should leave all other bitstring phases unchanged.
identity_phase_factor = 1
target_phase_factor = -1
bitstring_map = {}
for bitstring in product(bit, repeat=3):
    bitstring = "".join(bitstring)
    if bitstring == target_bitstring:
        bitstring_map[bitstring] = target_phase_factor
    else:
        bitstring_map[bitstring] = identity_phase_factor

We'll make sure the bistring_map is what we expected.

In [ ]:
for k,v in bitstring_map.iteritems():
    if k == target_bitstring:
        assert v == target_phase_factor, "The bitstring_map gives the target the wrong phase."
    else:
        assert v == identity_phase_factor, "The bitstring_map gives a non-target bitstring the wrong phase."

To use Grover's algorithm on quantum hardware we need to define a connection to a QVM or QPU. However we don't have a real connection in this notebook, so we mock out the response. If you intend to run this notebook on a QVM or QPU, ensure to replace `cxn` with a pyQuil connection object.

In [ ]:
with patch("pyquil.api.SyncConnection") as cxn:
    cxn.run_and_measure.return_value = (np.asarray([0, 1, 0], dtype=int), )

Now let's run Simon's algorithm. We instantiate the Simon object and then call its `find_mask` mehtod with the connection object and the 2-to-1 function whose mask we wish to find.
Finally we assert its correctness by checking with the mask we used to generate the function

In [ ]:
grover = Grover()
found_bistring = grover.find_bitstring(cxn, bitstring_map)
assert found_bistring == target_bitstring, "Found bitstring is not the expected bitstring."

If the assertion succeeds we know we found the correct bitstring.